In [3]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime
import time



def scrapper_1_instancia(progressivo=True):

    db.close_old_connections()


    def calculo_cnj(seq, ano, j, tr, vara):
        vara = str(vara)
        while len(vara) < 4:
            vara = '0' + vara
        n = int(f'{seq}{ano}{j}{tr}{vara}00')
        d = str(98 - n%97)
        if len(d) == 1:
            d = '0' + d
        seq = str(seq)
        while len(seq) < 7:
            seq = '0' + seq
        return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


    driver = Chrome()
    driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
    driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
    driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
    driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
    driver.get('https://esaj.tjsp.jus.br/cpopg/open.do')


    for v in [90]:

        pacote = 1000

        if PrimeiraInstancia.objects.filter(v=v).exists():
            ano = PrimeiraInstancia.objects.filter(v=v).latest('ano').ano
        else:
            ano = 2019

        if PrimeiraInstancia.objects.filter(ano=ano, v=v, seq__gt=100000).exists():
            q = PrimeiraInstancia.objects.filter(ano=ano, v=v, seq__gt=100000).order_by('-seq').only('seq')[:pacote]
            q = [i.seq for i in q]
            SEQUENCIAS_MAIOR = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
        else:
            SEQUENCIAS_MAIOR = [i for i in range(1000000, 1000000 + pacote*2)]

        if PrimeiraInstancia.objects.filter(ano=ano, v=v, seq__lt=1000000).exists():
            q = PrimeiraInstancia.objects.filter(ano=ano, v=v, seq__lt=1000000).order_by('-seq').only('seq')[:pacote]
            q = [i.seq for i in q]
            SEQUENCIAS_MENOR = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
        else:
            SEQUENCIAS_MENOR = [i for i in range(1, pacote*2)]


        for SEQUENCIAS in (SEQUENCIAS_MENOR, SEQUENCIAS_MAIOR):
            vazio = 0

            for seq in SEQUENCIAS:

                params = {
                'conversationId': '',
                'dadosConsulta.localPesquisa.cdLocal': '-1',
                'cbPesquisa': 'NUMPROC',
                'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
                'dadosConsulta.valorConsultaNuUnificado': calculo_cnj(seq, ano, 8, 26, v),
                'dadosConsulta.valorConsulta': '',
                'uuidCaptcha':'',}

                r = driver.request('GET', 'https://esaj.tjsp.jus.br/cpopg/search.do', params=params)

                res = {'seq': seq,
                       'ano': ano,
                       'v': v,
                       'info_url' : r.url}

                sp = bs(r.content, 'html5lib')

                res['nuProcessoUnificadoFormatado'] = sp.find(attrs={'id':'nuProcessoUnificadoFormatado'}).get('value')


                if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
                    vazio +=1

                else:
                    vazio = 0

                    if "mostrar: 'true'" in str(sp):
                        res['segredo_de_justica'] = True

                    elif sp.find(attrs={'class':'resultadoPaginacao'}):
                        res['problema'] = True

                    else:
                        if sp.find('a', attrs={'title': 'Pasta digital'}) is not None or sp.find('a', attrs={'class': 'linkConsultaSG'}) is not None:
                            res['digital'] = True

                        for item in sp.findAll(
                                attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
                            if item.get('for') == 'tag.dados.rotulo.principal':
                                res['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

                            elif item.get_text() == 'Distribuição:':
                                res[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

                                rp = item.findNext('span').findNext()
                                if rp.find('span') is not None:
                                    res['vara'] = rp.find('span').get_text().strip()

                            elif item.get_text() == 'Apensado ao:':
                                res[item.get_text().strip().replace(':', '')] = item.findNext(
                                    attrs={'class': 'processoPaiApenso'}).get_text().strip()

                            elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
                                res['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
                            else:
                                res[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

                            if item.get_text() == 'Recebido em:':
                                res['vara'] = item.findNext('span').findNext('span').get_text()

                        for i in sp.findAll('span', attrs={'class': 'labelClass'}):
                            res[i.get_text().replace(':', '')] = i.next.next.strip()

                        # extinto_suspenso
                        if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
                            res['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

                        if 'Distribuição' in res.keys():
                            rp = res['Distribuição'][:10]
                            res['data_inicial'] = datetime.datetime.strptime(rp, '%d/%m/%Y').date()
                        elif 'Recebido em' in res.keys():
                            rp = res['Recebido em'][:10]
                            res['data_inicial'] = datetime.datetime.strptime(rp, '%d/%m/%Y').date()

                        if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
                            res['tramitacao_prioritaria'] = 'Tramitação prioritária'
                        if 'Valor da ação' in res.keys():
                            res['Valor da ação'] = re.sub(r'\s\s+', ' ', res['Valor da ação'])
                            if re.match(r'(.*?)\s', res['Valor da ação']) is not None:
                                res['moeda'] = re.match(r'(.*?)\s', res['Valor da ação']).group(1)
                            if re.search(r'\s(.*)', res['Valor da ação']) is not None:
                                rp = re.search(r'\s(.*)', res['Valor da ação']).group(1).strip()
                                res['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

                            del res['Valor da ação']

                        if sp.find(attrs={'id':'tablePartesPrincipais'}).find('tbody'):
                            coleta = []
                            for entrada in sp.find(attrs={'id':'tablePartesPrincipais'}).find('tbody').findAll('tr'):
                                coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                                               'nome': entrada.findAll('td')[1].next.strip()})
                            if len(coleta) > 0:
                                res['partes'] = coleta

                        coleta = []
                        for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
                            linhas = entrada.findAll('td')
                            rp = {'ordem': _}
                            rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

                            if linhas[1].find('a'):
                                rp['link0'] = linhas[1].find('a').get('href')

                            if linhas[2].find('a'):
                                rp['link1'] = linhas[2].find('a').get('href')
                                rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
                            else:
                                rp['movimentacaotag'] = linhas[2].next.strip()

                            rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

                            rp['movimentacaodescricao'] = linhas[2].find('span').get_text().strip()
                            coleta.append(rp)
                        if len(coleta)>0:
                            res['movimentacoes'] = coleta

                res = {unidecode(k.lower().strip().replace(' ', '_')):v for k,v in res.items()}


                PrimeiraInstancia_m = PrimeiraInstancia(**{k:v for k,v in res.items() if k not in ['partes', 'movimentacoes' ]})
                PrimeiraInstancia_m.save()

                if 'partes' in res.keys():
                    partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]

                    for p in partes:
                        p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                        posicao = Posicao(primeira_instancia=PrimeiraInstancia_m, parte=p[0][0], posicao=p[1])
                        posicao.save()

                if 'movimentacoes' in res.keys():
                    m = [Movimentacao(**i, primeira_instancia=PrimeiraInstancia_m) for i in res['movimentacoes']]
                    Movimentacao.objects.bulk_create(m)

                if vazio == int(pacote*1.1):
                    break

                print(v, seq, vazio)


90 1 1
90 2 0
90 3 0
90 4 0
90 5 0
90 6 0
90 7 0
90 8 0
90 9 0
90 10 0
90 11 0
90 12 0
90 13 0
90 14 0
90 15 0
90 16 0
90 17 0
90 18 0
90 19 0
90 20 0
90 21 0
90 22 0
90 23 0
90 24 0
90 25 0
90 26 0
90 27 0
90 28 0
90 29 0
90 30 0
90 31 0
90 32 0
90 33 0
90 34 0
90 35 0
90 36 0
90 37 0
90 38 0
90 39 0
90 40 0
90 41 0
90 42 0
90 43 0
90 44 0
90 45 0
90 46 0
90 47 0
90 48 0
90 49 0
90 50 0
90 51 0
90 52 0
90 53 0
90 54 0
90 55 0
90 56 0
90 57 0
90 58 0
90 59 0
90 60 0
90 61 0
90 62 0
90 63 0
90 64 0
90 65 0
90 66 0
90 67 0
90 68 0
90 69 0
90 70 0
90 71 0
90 72 0
90 73 0
90 74 0
90 75 0
90 76 0
90 77 0
90 78 0
90 79 0
90 80 0
90 81 0
90 82 0
90 83 0
90 84 0
90 85 1
90 86 0
90 87 0
90 88 0
90 89 0
90 90 0
90 91 0
90 92 0
90 93 0
90 94 0
90 95 0
90 96 0
90 97 0
90 98 0
90 99 0
90 100 0
90 101 0
90 102 0
90 103 0
90 104 0
90 105 0
90 106 0
90 107 0
90 108 0
90 109 0
90 110 1
90 111 0
90 112 0
90 113 0
90 114 0
90 115 0
90 116 0
90 117 0
90 118 0
90 119 0
90 120 0
90 121 0
90 122 0
90 123 0
9

KeyboardInterrupt: 

In [18]:
db.close_old_connections()

PrimeiraInstancia.objects.all().delete()

(13, {'core.Movimentacao': 0, 'core.Posicao': 0, 'core.PrimeiraInstancia': 13})

In [16]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime
import time

ano = 2019
pacote = 500
PrimeiraInstancia.objects.filter(ano=ano, seq__gt=100000).exists()

if PrimeiraInstancia.objects.filter(ano=ano).exists():
    if PrimeiraInstancia.objects.filter(ano=ano, seq__gt=100000).exists():
        q = PrimeiraInstancia.objects.filter(ano=ano, seq_gt=100000).order_by('-seq').only('seq')[:pacote]
        q = [i.seq for i in q]
        SEQUENCIAS_MAIOR = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
    else:
        SEQUENCIAS_MAIOR = [i for i in range(1000000, 1000000 + pacote*2)]
    
    if PrimeiraInstancia.objects.filter(ano=ano, seq__lt=1000000):
        q = PrimeiraInstancia.objects.filter(ano=ano, seq__lt=1000000).order_by('-seq').only('seq')[:pacote]
        q = [i.seq for i in q]
        SEQUENCIAS_MENOR = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
    else:
        SEQUENCIAS_MENOR = [i for i in range(1, pacote*2)]



[1000000,
 1000001,
 1000002,
 1000003,
 1000004,
 1000005,
 1000006,
 1000007,
 1000008,
 1000009,
 1000010,
 1000011,
 1000012,
 1000013,
 1000014,
 1000015,
 1000016,
 1000017,
 1000018,
 1000019,
 1000020,
 1000021,
 1000022,
 1000023,
 1000024,
 1000025,
 1000026,
 1000027,
 1000028,
 1000029,
 1000030,
 1000031,
 1000032,
 1000033,
 1000034,
 1000035,
 1000036,
 1000037,
 1000038,
 1000039,
 1000040,
 1000041,
 1000042,
 1000043,
 1000044,
 1000045,
 1000046,
 1000047,
 1000048,
 1000049,
 1000050,
 1000051,
 1000052,
 1000053,
 1000054,
 1000055,
 1000056,
 1000057,
 1000058,
 1000059,
 1000060,
 1000061,
 1000062,
 1000063,
 1000064,
 1000065,
 1000066,
 1000067,
 1000068,
 1000069,
 1000070,
 1000071,
 1000072,
 1000073,
 1000074,
 1000075,
 1000076,
 1000077,
 1000078,
 1000079,
 1000080,
 1000081,
 1000082,
 1000083,
 1000084,
 1000085,
 1000086,
 1000087,
 1000088,
 1000089,
 1000090,
 1000091,
 1000092,
 1000093,
 1000094,
 1000095,
 1000096,
 1000097,
 1000098,
 1000099,


In [3]:
driver.get(r.url)

In [32]:
PrimeiraInstancia_m = PrimeiraInstancia(**{k:v for k,v in res.items() if k not in ['partes', 'movimentacoes' ]})
PrimeiraInstancia_m.save()

if 'partes' in res.keys():
    partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]

    for p in partes:
        p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
        posicao = Posicao(primeira_instancia=PrimeiraInstancia_m, parte=p[0][0], posicao=p[1])
        posicao.save()

if 'movimentacoes' in res.keys():
    m = [Movimentacao(**i, primeira_instancia=PrimeiraInstancia_m) for i in res['movimentacoes']]
    Movimentacao.objects.bulk_create(m)


In [16]:
coleta = []
for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
    linhas = entrada.findAll('td')
    rp = {'ordem': _}
    rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

    if linhas[1].find('a'):
        rp['link0'] = linhas[1].find('a').get('href')

    if linhas[2].find('a'):
        rp['link1'] = linhas[2].find('a').get('href')
        rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
    else:
        rp['movimentacaotag'] = linhas[2].next.strip()

    rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

    rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
    coleta.append(rp)
if len(coleta)>0:
    res['movimentacoes'] = coleta
    
coleta

[{'ordem': 0,
  'data': datetime.date(2019, 10, 31),
  'movimentacaotag': <MovimentacaoTag: Início da Execução Juntado>,
  'movimentacaodescricao': '\n\t\t\tProcesso principal: 1050915-47.2018.8.26.0053\n\t\t'},
 {'ordem': 1,
  'data': datetime.date(2019, 10, 31),
  'movimentacaotag': <MovimentacaoTag: Conclusos para Despacho>,
  'movimentacaodescricao': '\n\t\t\t\n\t\t'},
 {'ordem': 2,
  'data': datetime.date(2019, 10, 31),
  'link0': '/cpopg/abrirDocumentoVinculadoMovimentacao.do?processo.codigo=1H000GBQY0000&cdDocumento=74509505&nmRecursoAcessado=Despacho',
  'link1': '/cpopg/abrirDocumentoVinculadoMovimentacao.do?processo.codigo=1H000GBQY0000&cdDocumento=74509505&nmRecursoAcessado=Despacho',
  'movimentacaotag': <MovimentacaoTag: Despacho>,
  'movimentacaodescricao': '\n\t\t\tVistos. Preenchidos os requisitos do art. 534 do CPC, intime-se o executado, na pessoa do seu representante judicial, para que, querendo, apresente impugnação ao valor de R$5.269,11, atualizado para 22.10.2019

In [21]:
def get_tjsp(s, res):

    res['versao_parser'] = 5.1

    params = {'cbPesquisa': 'NUMPROC',
              'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
              'dadosConsulta.valorConsultaNuUnificado': res['nuprocessounificadoformatado']}

    r = s.get('https://esaj.tjsp.jus.br/cpopg/search.do', params=params, verify=False)
    sp = bs(r.content, 'html5lib')

    res['versao_parser'] = 5.1

    if sp.find('a', attrs={'title': 'Pasta digital'}) is not None or sp.find('a', attrs={'class': 'linkConsultaSG'}) is not None:

        res['digital'] = True

        crit = re.compile(r'requestScope = (.*?);\s+?var', re.I | re.DOTALL)
        parametro = re.search(r'.codigo=(.*?)&', r.url).group(1)

        s.get('https://esaj.tjsp.jus.br/cpopg/show.do?processo.foro=53&processo.codigo=%s&gateway=true' % parametro)

        r = s.get('https://esaj.tjsp.jus.br/cpopg/abrirPastaDigital.do?processo.codigo=%s' % parametro, verify=False)

        j = json.loads(re.search(crit, r.text).group(1))

        coleta = []

        c_assinatura = re.compile(
            'p\s{,4}a\s{,4}r\s{,4}a\s{,4}c\s{,4}o\s{,4}n\s{,4}f\s{,4}e\s{,4}r\s{,4}i\s{,4}r\s{,4}.*?fls.{,4}\d+',
            re.I | re.DOTALL)

        for linha in j:
            for sublinha in linha['children']:
                rp = {
                    'indicepagina': sublinha['data']['indicePagina'],
                    'title': linha['data']['title'],
                    'cddocumento': linha['data']['cdDocumento'],
                    'dtinclusao': datetime.datetime.strptime(linha['data']['dtInclusao'], '%d/%m/%Y %H:%M:%S'),
                    'cdtipodocdigital': linha['data']['cdTipoDocDigital'],
                    'parametros': sublinha['data']['parametros'].replace('amp;', ''),
                }

                texto_doc = s.get('https://esaj.tjsp.jus.br/pastadigital/getPDF.do?' + rp['parametros'], verify=False)

                while True:
                    try:
                        texto_doc = parser.from_buffer(io.BytesIO(texto_doc.content))['content']
                        break
                    except:
                        pass

                if texto_doc:
                    texto_doc = re.sub(c_assinatura, '', texto_doc).strip()

                    if len(texto_doc) > 100 and texto_doc[:7] != 'Atenção':
                        rp['conteudo'] = texto_doc.encode(encoding='latin1', errors='ignore').decode(encoding='latin1')

                coleta += [rp]

        if len(coleta) > 0:
            res['documentos'] = coleta

    if "mostrar: 'true'" in str(sp):
        res['segredo_de_justica'] = True
        return res


    # diversos

    for item in sp.findAll(
            attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
        if item.get('for') == 'tag.dados.rotulo.principal':
            res['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

        elif item.get_text() == 'Distribuição:':
            res[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

            rp = item.findNext('span').findNext()
            if rp.find('span') is not None:
                res['vara'] = rp.find('span').get_text().strip()

        elif item.get_text() == 'Apensado ao:':
            res[item.get_text().strip().replace(':', '')] = item.findNext(
                attrs={'class': 'processoPaiApenso'}).get_text().strip()

        elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
            res['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
        else:
            res[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

        if item.get_text() == 'Recebido em:':
            res['vara'] = item.findNext('span').findNext('span').get_text()

    for i in sp.findAll('span', attrs={'class': 'labelClass'}):
        res[i.get_text().replace(':', '')] = i.next.next.strip()

    # extinto_suspenso
    if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
        res['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

    if 'Distribuição' in res.keys():
        rp = res['Distribuição'][:10]
        res['data_inicial'] = datetime.datetime.strptime(rp, '%d/%m/%Y').date()
    elif 'Recebido em' in res.keys():
        rp = res['Recebido em'][:10]
        res['data_inicial'] = datetime.datetime.strptime(rp, '%d/%m/%Y').date()

    if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
        res['tramitacao_prioritaria'] = 'Tramitação prioritária'
    if 'Valor da ação' in res.keys():
        res['Valor da ação'] = re.sub(r'\s\s+', ' ', res['Valor da ação'])
        if re.match(r'(.*?)\s', res['Valor da ação']) is not None:
            res['moeda'] = re.match(r'(.*?)\s', res['Valor da ação']).group(1)
        if re.search(r'\s(.*)', res['Valor da ação']) is not None:
            rp = re.search(r'\s(.*)', res['Valor da ação']).group(1).strip()
            res['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

        del res['Valor da ação']

    # partes
    coleta = []
    controle = []

    for item in sp.findAll('td', attrs={'align': 'right', 'width': '141', 'valign': 'top',
                                        'style': 'padding-bottom: 5px'}):
        rp = {}

        if item.findNext('td').next.strip() not in controle:
            controle += [item.findNext('td').next.strip()]

            rp['nome'] = item.findNext('td').next.strip()

            rp['posicao'] = item.get_text().strip().replace(':', '')

            procuradores = []

            for subitem in item.findNextSibling().findAll('span'):
                classe = subitem.get_text().replace(':', '').strip()
                nome = subitem.next.next.strip()
                procuradores += [[classe, nome]]

            classes = list(set([i[0] for i in procuradores]))
            subcoleta = {}

            for classe in classes:
                nome_procurador = [i[1] for i in procuradores if i[0] == classe]
                subcoleta[classe] = nome_procurador

            if len(subcoleta) > 0:
                rp['procuradores'] = subcoleta
            coleta += [rp]

    if len(coleta) > 0:
        res['partes'] = coleta

    # movimentacoes
    coleta = []
    lista = sp.findAll('td', attrs={'width': '120', 'style': 'vertical-align: top'})
    r = list(range(len(lista) - 1, -1, -1))

    for n, item in zip(r, lista):
        entrada = {'ordem': n,
                   'data': datetime.datetime.strptime(item.get_text().strip(), '%d/%m/%Y').date()}

        if len(item.findNext('td').findChildren()) > 0:
            entrada['link'] = item.findNext('td').findNext('td').findChild('a').get('id')
            entrada['tag'] = item.findNext('td').findNext('td').findChild('a').get_text().strip()
        else:

            entrada['tag'] = item.findNext('td').findNext('td').next.strip()
            entrada['descricao'] = item.findNext('td').findNext('td').findChild('span').get_text().strip()
        coleta += [entrada]

    if len(coleta) > 0:
        res['movimentacoes'] = coleta

    res = {unidecode(k.lower()).strip().replace(' ', '_'): v for k, v in res.items() if v}

    docinformacao = Informacoes(**{k: v for k, v in res.items() if k not in ['movimentacoes', 'partes', 'documentos']})

    if not docinformacao.full_clean():
        docinformacao.save()

        if 'documentos' in res.keys():
            for entrada_doc in res['documentos']:
                docDocumeto = Documentos(**entrada_doc, informacao=docinformacao)
                docDocumeto.save()

        if 'partes' in res.keys():
            for entrada_doc in res['partes']:
                docParte = Partes(**entrada_doc, informacao=docinformacao)
                docParte.save()

        if 'movimentacoes' in res.keys():
            for entrada_doc in res['movimentacoes']:
                docMovimentacao = Movimentacoes(**entrada_doc, informacao=docinformacao)
                docMovimentacao.save()

    return False

{'seq': 2262011,
 'info_url': 'https://esaj.tjsp.jus.br/cposg/search.do?conversationId=&paginaConsulta=1&localPesquisa.cdLocal=-1&cbPesquisa=NUMPROC&tipoNuProcesso=UNIFICADO&dePesquisaNuUnificado=2262011-86.2019.8.26.0000&dePesquisa=&uuidCaptcha=',
 'segredo_de_justica': True}

In [19]:
def calculo_cnj(seq, ano, j, tr, vara):
    vara = str(vara)
    while len(vara) < 4:
        vara = vara + '0'
    n = int(f'{seq}{ano}{j}{tr}{vara}00')
    d = str(98 - n%97)
    if len(d) == 1:
        d = '0' + d
    seq = str(seq)
    while len(seq) < 7:
        seq = '0' + seq
    return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


driver = Chrome()

driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

vazio = 0

if SegundaInstancia.objects.all().exists():
    seq = SegundaInstancia.objects.latest('seq').seq + 1
else:
    seq = 2262000

while True:

    params = {

    'conversationId': '',
    'paginaConsulta': '1',
    'localPesquisa.cdLocal': '-1',
    'cbPesquisa': 'NUMPROC',
    'tipoNuProcesso': 'UNIFICADO',
    'dePesquisaNuUnificado': calculo_cnj(seq, 2019, 8, 26, 0),
    'dePesquisa': '',
    'uuidCaptcha': '',
    }

    r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)

    res = {'seq': seq,
           'info_url' : r.url}

    sp = bs(r.content, 'html5lib')

    if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
        vazio +=1

    else:
        vazio = 0
        if sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
            res['segredo_de_justica'] = True
        else:
            tabela_dados_gerais = sp.findAll(attrs={'class':'secaoFormBody'})[1].find('tbody')
            for entrada in tabela_dados_gerais.findAll('tr'):
                try:
                    k = entrada.find('label').get_text()
                    v = entrada.find('span').get_text()
                    res[k] = v
                except:
                    pass

            tabela_1_instancia = sp.find(text=re.compile('Números de 1ª Instância')).findAllNext('table')[1].find('tbody')
            if tabela_1_instancia:
                coleta = []
                for entrada in tabela_1_instancia.findAll('tr'):
                    coleta.append({k: v.get_text().strip() for k,v in zip(['n_primeira_instancia', 'foro', 'vara', 'juiz', 'obs'], entrada.findAll('td')) })    
                if len(coleta) > 0:
                    res['numeros_1_instancia'] = coleta

            tabela_partes = sp.find(text=re.compile('Partes do Processo')).findNext('table').find('tbody')

            coleta = []
            for entrada in tabela_partes.findAll('tr'):
                coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                               'nome': entrada.findAll('td')[1].next.strip()})
            if len(coleta) > 0:
                res['partes'] = coleta

        coleta = []
        for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
            linhas = entrada.findAll('td')
            rp = {'ordem': _}
            rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

            if linhas[1].find('a'):
                rp['link0'] = linhas[1].find('a').get('href')

            if linhas[2].find('a'):
                rp['link1'] = linhas[2].find('a').get('href')
                rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
            else:
                rp['movimentacaotag'] = linhas[2].next.strip()

            rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

            rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
            coleta.append(rp)
        if len(coleta)>0:
            res['movimentacoes'] = coleta

        res = {k.replace(':','').lower().strip(): v for k,v in res.items()}
        res = {unidecode(k).replace(' ','_').replace('_/_','_'): (v.strip() if isinstance(v, str) else v) for k,v in res.items()}

        if 'valor_da_acao' in res.keys():
            res['valor_da_acao'] = float(res['valor_da_acao'].replace('.','').replace(',','.'))

        SegundaInstancia_m = SegundaInstancia(**{k:v for k,v in res.items() if k not in ['partes','numeros_1_instancia', 'movimentacoes' ]})
        SegundaInstancia_m.save()

        if 'partes' in res.keys():
            partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]

            for p in partes:
                p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                posicao = Posicao(segunda_instancia=SegundaInstancia_m, parte=p[0][0], posicao=p[1])
                posicao.save()

        if 'numeros_1_instancia' in res.keys():
            n_1_instancia = [{k:v for k,v in i.items() if v !='-'} for i in res['numeros_1_instancia']]
            Numero1Instancia.objects.bulk_create([Numero1Instancia(**i, segunda_instancia=SegundaInstancia_m) for i in n_1_instancia])

        if 'movimentacoes' in res.keys():
            m = [Movimentacao(**i, segunda_instancia=SegundaInstancia_m) for i in res['movimentacoes']]
            Movimentacao.objects.bulk_create(m)

    seq += 1
    if vazio == 25:
        break

    print(seq, vazio)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [11]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime

db.close_old_connections()

def calculo_cnj(seq, ano, j, tr, vara):
    vara = str(vara)
    while len(vara) < 4:
        vara = vara + '0'
    n = int(f'{seq}{ano}{j}{tr}{vara}00')
    d = str(98 - n%97)
    if len(d) == 1:
        d = '0' + d
    seq = str(seq)
    while len(seq) < 7:
        seq = '0' + seq
    return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


driver = Chrome()

driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

vazio = 0

if SegundaInstancia.objects.all().exists():
    seq = SegundaInstancia.objects.latest('seq').seq + 1
else:
    seq = 2262000

while True:

    params = {

    'conversationId': '',
    'paginaConsulta': '1',
    'localPesquisa.cdLocal': '-1',
    'cbPesquisa': 'NUMPROC',
    'tipoNuProcesso': 'UNIFICADO',
    'dePesquisaNuUnificado': calculo_cnj(seq, 2019, 8, 26, 0),
    'dePesquisa': '',
    'uuidCaptcha': '',
    }

    r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)

    res = {'seq': seq,
           'info_url' : r.url}

    sp = bs(r.content, 'html5lib')

    if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
        vazio +=1

    else:
        vazio = 0
        if sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
            res['segredo_de_justica'] = True
        else:
            tabela_dados_gerais = sp.findAll(attrs={'class':'secaoFormBody'})[1].find('tbody')
            for entrada in tabela_dados_gerais.findAll('tr'):
                try:
                    k = entrada.find('label').get_text()
                    v = entrada.find('span').get_text()
                    res[k] = v
                except:
                    pass

            tabela_1_instancia = sp.find(text=re.compile('Números de 1ª Instância')).findAllNext('table')[1].find('tbody')
            if tabela_1_instancia:
                coleta = []
                for entrada in tabela_1_instancia.findAll('tr'):
                    coleta.append({k: v.get_text().strip() for k,v in zip(['n_primeira_instancia', 'foro', 'vara', 'juiz', 'obs'], entrada.findAll('td')) })    
                if len(coleta) > 0:
                    res['numeros_1_instancia'] = coleta

            tabela_partes = sp.find(text=re.compile('Partes do Processo')).findNext('table').find('tbody')

            coleta = []
            for entrada in tabela_partes.findAll('tr'):
                coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                               'nome': entrada.findAll('td')[1].next.strip()})
            if len(coleta) > 0:
                res['partes'] = coleta
                
        coleta = []
        for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
            linhas = entrada.findAll('td')
            rp = {'ordem': _}
            rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

            if linhas[1].find('a'):
                rp['link0'] = linhas[1].find('a').get('href')

            if linhas[2].find('a'):
                rp['link1'] = linhas[2].find('a').get('href')
                rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
            else:
                rp['movimentacaotag'] = linhas[2].next.strip()

            rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

            rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
            coleta.append(rp)
        if len(coleta)>0:
            res['movimentacoes'] = coleta

        res = {k.replace(':','').lower().strip(): v for k,v in res.items()}
        res = {unidecode(k).replace(' ','_').replace('_/_','_'): (v.strip() if isinstance(v, str) else v) for k,v in res.items()}
        
        if 'valor_da_acao' in res.keys():
            res['valor_da_acao'] = float(res['valor_da_acao'].replace('.','').replace(',','.'))

        SegundaInstancia_m = SegundaInstancia(**{k:v for k,v in res.items() if k not in ['partes','numeros_1_instancia', 'movimentacoes' ]})
        SegundaInstancia_m.save()

        if 'partes' in res.keys():
            partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]
            
            for p in partes:
                p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                posicao = Posicao(segunda_instancia=SegundaInstancia_m, parte=p[0][0], posicao=p[1])
                posicao.save()

        if 'numeros_1_instancia' in res.keys():
            n_1_instancia = [{k:v for k,v in i.items() if v !='-'} for i in res['numeros_1_instancia']]
            Numero1Instancia.objects.bulk_create([Numero1Instancia(**i, segunda_instancia=SegundaInstancia_m) for i in n_1_instancia])
        
        if 'movimentacoes' in res.keys():
            m = [Movimentacao(**i, segunda_instancia=SegundaInstancia_m) for i in res['movimentacoes']]
            Movimentacao.objects.bulk_create(m)

    seq += 1
    if vazio == 25:
        break

    print(seq, vazio)
    
    break


2202004 0


In [10]:
res['movimentacoes']


In [5]:
# driver.get(r.url)
# sp = bs(r.content, 'html5lib')
coleta = []
for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
    linhas = entrada.findAll('td')
    rp = {'ordem': _}
    rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()
    
    if linhas[1].find('a'):
        rp['link0'] = linhas[1].find('a').get('href')
    
    if linhas[2].find('a'):
        rp['link1'] = linhas[2].find('a').get('href')
        rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
    else:
        rp['movimentacaotag'] = linhas[2].next.strip()
    
    rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])
    
    rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
    coleta.append(rp)
    
coleta

[{'ordem': 0,
  'data': datetime.date(2019, 9, 10),
  'movimentacaotag': <MovimentacaoTag: Processo Cadastrado>,
  'movimentacaodescricao': 'SJ 1.2.1 - Serv. de Entrada de Originários do Orgão Especial e Câmara Especial'},
 {'ordem': 1,
  'data': datetime.date(2019, 9, 10),
  'movimentacaotag': <MovimentacaoTag: Processo encaminhado para a Distribuição de Originários>,
  'movimentacaodescricao': ''},
 {'ordem': 2,
  'data': datetime.date(2019, 9, 10),
  'movimentacaotag': <MovimentacaoTag: Distribuição por Sorteio>,
  'movimentacaodescricao': 'Órgão Julgador: 102 - Órgão Especial\nRelator: 10549 - Beretta da Silveira'},
 {'ordem': 3,
  'data': datetime.date(2019, 9, 10),
  'link0': '#?cdDocumento=5',
  'link1': '#?cdDocumento=5',
  'movimentacaotag': <MovimentacaoTag: Conclusos para o Relator (Expedido Termo com Conclusão)>,
  'movimentacaodescricao': 'BERETTA DA SILVEIRA'},
 {'ordem': 4,
  'data': datetime.date(2019, 9, 11),
  'link0': '#?cdDocumento=6',
  'link1': '#?cdDocumento=6',


In [12]:
SegundaInstancia.objects.all().delete()

(38,
 {'core.Numero1Instancia': 4,
  'core.Movimentacao': 21,
  'core.Posicao': 9,
  'core.SegundaInstancia': 4})

In [19]:
from core.models import *


# a = Person(nome='Ciclano', pertinente=True)
# a.save()

# b = Processo(nome='Teste1')
# b.save()

# c = Posicao(person=a, processo=b)
# c.save()

# a = Processo.objects.latest('id')

a

<Processo: Teste1>

In [36]:
res

NameError: name 'res' is not defined

In [2]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
import concurrent.futures




# segunda_instancia.objects.all().delete()

(16949,
 {'core.numero_1_instancia': 3988,
  'core.partes': 8946,
  'core.segunda_instancia': 4015})

In [7]:

def calculo_cnj(seq, ano, j, tr, vara):
    vara = str(vara)
    while len(vara) < 4:
        vara = vara + '0'
    n = int(f'{seq}{ano}{j}{tr}{vara}00')
    d = str(98 - n%97)
    if len(d) == 1:
        d = '0' + d
    seq = str(seq)
    while len(seq) < 7:
        seq = '0' + seq
    return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'

import time

driver = Chrome()

driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

seq = 2000002


params = {

'conversationId': '',
'paginaConsulta': '1',
'localPesquisa.cdLocal': '-1',
'cbPesquisa': 'NUMPROC',
'tipoNuProcesso': 'UNIFICADO',
'dePesquisaNuUnificado': calculo_cnj(seq,2019,8,26,0),
'dePesquisa': '',
'uuidCaptcha': '',
}

r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)
driver.get(r.url)
seq +=1
time.sleep(0.6)



In [20]:
calculo_cnj(seq,2019,8,26,0)

'1288213-20.2019.8.26.0000'

In [15]:
2000523-17.2019.8.26.0000,

<td id="mensagemRetorno" role="alert" tabindex="0">
           <li>Não existem informações disponíveis para os parâmetros informados.</li>
       </td>